In [1]:
import os
import pandas as pd
import tweepy
import re
import string
import preprocessor as p
import datetime
import nltk
import numpy as np
import emoji
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from textblob import TextBlob
from preprocessor.api import clean, tokenize, parse
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer
from datetime import timedelta
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary

In [3]:
ds = pd.read_csv("perkuliahan.csv")
ds.head()

,username,tweetcreatedat,text
0,PijaMoris,2022-10-05 23:59:59+00:00,@fedtriyahya Selawat yg ustaz wadi anuar selal...
1,akaipinheel,2022-10-05 23:59:52+00:00,@collegemenfess siapa yang ngira dulu kalo mau...
2,buyny,2022-10-05 23:59:52+00:00,sebenernya aku gak kuat ya Allah kalo kuliah p...
3,bungwol,2022-10-05 23:59:22+00:00,YaAllah w yg dulu kuliah PP tangsel rawamangun...
4,newaccautoflbck,2022-10-05 23:59:18+00:00,ptn! kuliah di upnvj gapapa kan? https://t.co/...


# CLEANING

In [4]:
def remove_pattern(text, pattern_regex):
    r = re.findall(pattern_regex, text)
    for i in r:
         text = re.sub(i, '', text)
    return text

In [5]:
ds['clean_text'] = np.vectorize(remove_pattern)(ds['text'], " *RT* | *@[\w]*")
ds.head()

,username,tweetcreatedat,text,clean_text
0,PijaMoris,2022-10-05 23:59:59+00:00,@fedtriyahya Selawat yg ustaz wadi anuar selal...,Selawat yg ustaz wadi anuar selalu sebut dala...
1,akaipinheel,2022-10-05 23:59:52+00:00,@collegemenfess siapa yang ngira dulu kalo mau...,siapa yang ngira dulu kalo mau jadi polisi ha...
2,buyny,2022-10-05 23:59:52+00:00,sebenernya aku gak kuat ya Allah kalo kuliah p...,sebenernya aku gak kuat ya Allah kalo kuliah p...
3,bungwol,2022-10-05 23:59:22+00:00,YaAllah w yg dulu kuliah PP tangsel rawamangun...,YaAllah w yg dulu kuliah PP tangsel rawamangun...
4,newaccautoflbck,2022-10-05 23:59:18+00:00,ptn! kuliah di upnvj gapapa kan? https://t.co/...,ptn! kuliah di upnvj gapapa kan? https://t.co/...


In [6]:
def remove_symb(text):
    text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ", text).split())
    return text
ds['remove_symb'] = ds['clean_text'].apply(lambda x: remove_symb(x))
ds.head()

,username,tweetcreatedat,text,clean_text,remove_symb
0,PijaMoris,2022-10-05 23:59:59+00:00,@fedtriyahya Selawat yg ustaz wadi anuar selal...,Selawat yg ustaz wadi anuar selalu sebut dala...,Selawat yg ustaz wadi anuar selalu sebut dalam...
1,akaipinheel,2022-10-05 23:59:52+00:00,@collegemenfess siapa yang ngira dulu kalo mau...,siapa yang ngira dulu kalo mau jadi polisi ha...,siapa yang ngira dulu kalo mau jadi polisi har...
2,buyny,2022-10-05 23:59:52+00:00,sebenernya aku gak kuat ya Allah kalo kuliah p...,sebenernya aku gak kuat ya Allah kalo kuliah p...,sebenernya aku gak kuat ya Allah kalo kuliah p...
3,bungwol,2022-10-05 23:59:22+00:00,YaAllah w yg dulu kuliah PP tangsel rawamangun...,YaAllah w yg dulu kuliah PP tangsel rawamangun...,YaAllah w yg dulu kuliah PP tangsel rawamangun...
4,newaccautoflbck,2022-10-05 23:59:18+00:00,ptn! kuliah di upnvj gapapa kan? https://t.co/...,ptn! kuliah di upnvj gapapa kan? https://t.co/...,ptn kuliah di upnvj gapapa kan


In [7]:
def remove(text):
    #menghilangkan text RT
    text = re.sub(r'^RT[\s]+','', text)
    
    #menghilangkan text CM
    text = re.sub(r'^cm[\s]+','', text)
    
    #menghilangkan hashtag
    text = re.sub(r'#','', text)
    
    #menghilangkan angka
    text = re.sub(r'[0-9]+','', text)
    
    return text

ds['clean_text_two'] = ds['remove_symb'].apply(lambda x: remove(x))
ds.head()

,username,tweetcreatedat,text,clean_text,remove_symb,clean_text_two
0,PijaMoris,2022-10-05 23:59:59+00:00,@fedtriyahya Selawat yg ustaz wadi anuar selal...,Selawat yg ustaz wadi anuar selalu sebut dala...,Selawat yg ustaz wadi anuar selalu sebut dalam...,Selawat yg ustaz wadi anuar selalu sebut dalam...
1,akaipinheel,2022-10-05 23:59:52+00:00,@collegemenfess siapa yang ngira dulu kalo mau...,siapa yang ngira dulu kalo mau jadi polisi ha...,siapa yang ngira dulu kalo mau jadi polisi har...,siapa yang ngira dulu kalo mau jadi polisi har...
2,buyny,2022-10-05 23:59:52+00:00,sebenernya aku gak kuat ya Allah kalo kuliah p...,sebenernya aku gak kuat ya Allah kalo kuliah p...,sebenernya aku gak kuat ya Allah kalo kuliah p...,sebenernya aku gak kuat ya Allah kalo kuliah p...
3,bungwol,2022-10-05 23:59:22+00:00,YaAllah w yg dulu kuliah PP tangsel rawamangun...,YaAllah w yg dulu kuliah PP tangsel rawamangun...,YaAllah w yg dulu kuliah PP tangsel rawamangun...,YaAllah w yg dulu kuliah PP tangsel rawamangun...
4,newaccautoflbck,2022-10-05 23:59:18+00:00,ptn! kuliah di upnvj gapapa kan? https://t.co/...,ptn! kuliah di upnvj gapapa kan? https://t.co/...,ptn kuliah di upnvj gapapa kan,ptn kuliah di upnvj gapapa kan


In [8]:
ds.drop_duplicates(subset = "clean_text_two", keep = 'first', inplace = True)
ds.head()

,username,tweetcreatedat,text,clean_text,remove_symb,clean_text_two
0,PijaMoris,2022-10-05 23:59:59+00:00,@fedtriyahya Selawat yg ustaz wadi anuar selal...,Selawat yg ustaz wadi anuar selalu sebut dala...,Selawat yg ustaz wadi anuar selalu sebut dalam...,Selawat yg ustaz wadi anuar selalu sebut dalam...
1,akaipinheel,2022-10-05 23:59:52+00:00,@collegemenfess siapa yang ngira dulu kalo mau...,siapa yang ngira dulu kalo mau jadi polisi ha...,siapa yang ngira dulu kalo mau jadi polisi har...,siapa yang ngira dulu kalo mau jadi polisi har...
2,buyny,2022-10-05 23:59:52+00:00,sebenernya aku gak kuat ya Allah kalo kuliah p...,sebenernya aku gak kuat ya Allah kalo kuliah p...,sebenernya aku gak kuat ya Allah kalo kuliah p...,sebenernya aku gak kuat ya Allah kalo kuliah p...
3,bungwol,2022-10-05 23:59:22+00:00,YaAllah w yg dulu kuliah PP tangsel rawamangun...,YaAllah w yg dulu kuliah PP tangsel rawamangun...,YaAllah w yg dulu kuliah PP tangsel rawamangun...,YaAllah w yg dulu kuliah PP tangsel rawamangun...
4,newaccautoflbck,2022-10-05 23:59:18+00:00,ptn! kuliah di upnvj gapapa kan? https://t.co/...,ptn! kuliah di upnvj gapapa kan? https://t.co/...,ptn kuliah di upnvj gapapa kan,ptn kuliah di upnvj gapapa kan


In [9]:
ds.shape

(375, 6)

# CASE FOLDING

In [10]:
ds['case_fold'] = ds['clean_text_two'].str.lower()

ds.head()

,username,tweetcreatedat,text,clean_text,remove_symb,clean_text_two,case_fold
0,PijaMoris,2022-10-05 23:59:59+00:00,@fedtriyahya Selawat yg ustaz wadi anuar selal...,Selawat yg ustaz wadi anuar selalu sebut dala...,Selawat yg ustaz wadi anuar selalu sebut dalam...,Selawat yg ustaz wadi anuar selalu sebut dalam...,selawat yg ustaz wadi anuar selalu sebut dalam...
1,akaipinheel,2022-10-05 23:59:52+00:00,@collegemenfess siapa yang ngira dulu kalo mau...,siapa yang ngira dulu kalo mau jadi polisi ha...,siapa yang ngira dulu kalo mau jadi polisi har...,siapa yang ngira dulu kalo mau jadi polisi har...,siapa yang ngira dulu kalo mau jadi polisi har...
2,buyny,2022-10-05 23:59:52+00:00,sebenernya aku gak kuat ya Allah kalo kuliah p...,sebenernya aku gak kuat ya Allah kalo kuliah p...,sebenernya aku gak kuat ya Allah kalo kuliah p...,sebenernya aku gak kuat ya Allah kalo kuliah p...,sebenernya aku gak kuat ya allah kalo kuliah p...
3,bungwol,2022-10-05 23:59:22+00:00,YaAllah w yg dulu kuliah PP tangsel rawamangun...,YaAllah w yg dulu kuliah PP tangsel rawamangun...,YaAllah w yg dulu kuliah PP tangsel rawamangun...,YaAllah w yg dulu kuliah PP tangsel rawamangun...,yaallah w yg dulu kuliah pp tangsel rawamangun...
4,newaccautoflbck,2022-10-05 23:59:18+00:00,ptn! kuliah di upnvj gapapa kan? https://t.co/...,ptn! kuliah di upnvj gapapa kan? https://t.co/...,ptn kuliah di upnvj gapapa kan,ptn kuliah di upnvj gapapa kan,ptn kuliah di upnvj gapapa kan


# FILTERING STOPWORDS

In [11]:
stopwords_bahasa = stopwords.words('indonesian')

stop_factory = StopWordRemoverFactory().get_stop_words()
add_list = [
    "yg", "dgn", "rt", "tdk", "jgn", "krn", "knp", "btw", "tp", "krn", "sdh", "gas", "ok", "oke", "ga", "jgn", "tdk", "si", 
    "sih", "gw", "gua", "gue", "lo", "lu", "jg", "nnti", "cuma", "ntar", "bsk", "kyk", "dl", "dulu", "kzl", "mls", "jls", "ksl",
    "byk", "bgs", "kl", "udh", "tlg", "bgt", "masi", "loe", "kmu", "cuy", "gt", "twt"
]

data_stopwords = stop_factory + add_list

dictionary = ArrayDictionary(data_stopwords)
str = StopWordRemover(dictionary)
#print(data_stopwords)

# TOKENIZING DAN STEMMING

In [12]:
factory_stem = StemmerFactory()
stemmer = factory_stem.create_stemmer()

happy_emoticons = set([
    ":-)", ":)", ":))", ":)))", ":))))", "<3", ":-D", ":D", ":')", ":>'", ":*", ":*)", ":-))", ":-DD" 
])

sad_emoticons = set([
    ":(", ":((", ":(((", ":'(",":-(", ":-((", ":-(((", ":-c", ":c", ":<"
])

emoticons = happy_emoticons.union(sad_emoticons)

def tokenize(text):
    tokenizer = TweetTokenizer(preserve_case = False, strip_handles = True, reduce_len = True)
    tweet_tokens = tokenizer.tokenize(text)
    
    tweet_tokenizer = []
    for word in tweet_tokens:
        if(word not in data_stopwords and 
           word not in emoticons and
           word not in string.punctuation):
            stem_word = stemmer.stem(word)
            tweet_tokenizer.append(stem_word)
    return tweet_tokenizer
ds['tokenize'] = ds['case_fold'].apply(lambda x:tokenize(x))
ds.head(50)

,username,tweetcreatedat,text,clean_text,remove_symb,clean_text_two,case_fold,tokenize
0,PijaMoris,2022-10-05 23:59:59+00:00,@fedtriyahya Selawat yg ustaz wadi anuar selal...,Selawat yg ustaz wadi anuar selalu sebut dala...,Selawat yg ustaz wadi anuar selalu sebut dalam...,Selawat yg ustaz wadi anuar selalu sebut dalam...,selawat yg ustaz wadi anuar selalu sebut dalam...,"[selawat, ustaz, wadi, anuar, kuliah, hengat, ..."
1,akaipinheel,2022-10-05 23:59:52+00:00,@collegemenfess siapa yang ngira dulu kalo mau...,siapa yang ngira dulu kalo mau jadi polisi ha...,siapa yang ngira dulu kalo mau jadi polisi har...,siapa yang ngira dulu kalo mau jadi polisi har...,siapa yang ngira dulu kalo mau jadi polisi har...,"[ngira, kalo, polisi, kuliah]"
2,buyny,2022-10-05 23:59:52+00:00,sebenernya aku gak kuat ya Allah kalo kuliah p...,sebenernya aku gak kuat ya Allah kalo kuliah p...,sebenernya aku gak kuat ya Allah kalo kuliah p...,sebenernya aku gak kuat ya Allah kalo kuliah p...,sebenernya aku gak kuat ya allah kalo kuliah p...,"[sebenernya, gak, kuat, allah, kalo, kuliah, p..."
3,bungwol,2022-10-05 23:59:22+00:00,YaAllah w yg dulu kuliah PP tangsel rawamangun...,YaAllah w yg dulu kuliah PP tangsel rawamangun...,YaAllah w yg dulu kuliah PP tangsel rawamangun...,YaAllah w yg dulu kuliah PP tangsel rawamangun...,yaallah w yg dulu kuliah pp tangsel rawamangun...,"[yaallah, kuliah, pp, tangsel, rawamangun, aje..."
4,newaccautoflbck,2022-10-05 23:59:18+00:00,ptn! kuliah di upnvj gapapa kan? https://t.co/...,ptn! kuliah di upnvj gapapa kan? https://t.co/...,ptn kuliah di upnvj gapapa kan,ptn kuliah di upnvj gapapa kan,ptn kuliah di upnvj gapapa kan,"[ptn, kuliah, upnvj, gapapa]"
5,tapibohoonk,2022-10-05 23:59:15+00:00,[cm] kalian sering kehilangan ambis gak sih pa...,[cm] kalian sering kehilangan ambis gak sih pa...,cm kalian sering kehilangan ambis gak sih pas ...,kalian sering kehilangan ambis gak sih pas kul...,kalian sering kehilangan ambis gak sih pas kul...,"[hilang, ambis, gak, pas, kuliah, kayak, gak, ..."
6,cutmybutter2fly,2022-10-05 23:59:14+00:00,TWITTER PLEASE DO YOUR MAGIC\n\nentah ini work...,TWITTEPLEASE DO YOUMAGIC\n\nentah ini work it ...,TWITTEPLEASE DO YOUMAGIC entah ini work it or ...,TWITTEPLEASE DO YOUMAGIC entah ini work it or ...,twitteplease do youmagic entah ini work it or ...,"[twitteplease, do, youmagic, work, it, or, not..."
7,bydamieee,2022-10-05 23:59:13+00:00,CLAP UDH ADA CHALLENGE NYA BLM SI??? PGN JOINN...,CLAP UDH ADA CHALLENGE NYA BLM SI??? PGN JOINN...,CLAP UDH ADA CHALLENGE NYA BLM SI PGN JOINNN S...,CLAP UDH ADA CHALLENGE NYA BLM SI PGN JOINNN S...,clap udh ada challenge nya blm si pgn joinnn s...,"[clap, challenge, blm, pgn, joinnn, sm, tmn, k..."
9,FukeinaKo,2022-10-05 23:58:53+00:00,Tidur jam 3. Bangun 6.20. Kuliah 6.30,Tidur jam 3. Bangun 6.20. Kuliah 6.30,Tidur jam 3 Bangun 6 20 Kuliah 6 30,Tidur jam Bangun Kuliah,tidur jam bangun kuliah,"[tidur, jam, bangun, kuliah]"
10,dibawahpressure,2022-10-05 23:58:44+00:00,Riiill. Dari aku SD sampe sekarang kuliah kebu...,Riiill. Dari aku SD sampe sekarang kuliah kebu...,Riiill Dari aku SD sampe sekarang kuliah kebun...,Riiill Dari aku SD sampe sekarang kuliah kebun...,riiill dari aku sd sampe sekarang kuliah kebun...,"[riiill, sd, sampe, kuliah, kebun, papa, modal]"


# Proses Kembalian

In [13]:
def remove_point(text):
    text = " ".join([char for char in text if char not in string.punctuation])
    return text
ds['data'] = ds['tokenize'].apply(lambda x: remove_point(x))
ds.head(50)

,username,tweetcreatedat,text,clean_text,remove_symb,clean_text_two,case_fold,tokenize,data
0,PijaMoris,2022-10-05 23:59:59+00:00,@fedtriyahya Selawat yg ustaz wadi anuar selal...,Selawat yg ustaz wadi anuar selalu sebut dala...,Selawat yg ustaz wadi anuar selalu sebut dalam...,Selawat yg ustaz wadi anuar selalu sebut dalam...,selawat yg ustaz wadi anuar selalu sebut dalam...,"[selawat, ustaz, wadi, anuar, kuliah, hengat, ...",selawat ustaz wadi anuar kuliah hengat aih nam...
1,akaipinheel,2022-10-05 23:59:52+00:00,@collegemenfess siapa yang ngira dulu kalo mau...,siapa yang ngira dulu kalo mau jadi polisi ha...,siapa yang ngira dulu kalo mau jadi polisi har...,siapa yang ngira dulu kalo mau jadi polisi har...,siapa yang ngira dulu kalo mau jadi polisi har...,"[ngira, kalo, polisi, kuliah]",ngira kalo polisi kuliah
2,buyny,2022-10-05 23:59:52+00:00,sebenernya aku gak kuat ya Allah kalo kuliah p...,sebenernya aku gak kuat ya Allah kalo kuliah p...,sebenernya aku gak kuat ya Allah kalo kuliah p...,sebenernya aku gak kuat ya Allah kalo kuliah p...,sebenernya aku gak kuat ya allah kalo kuliah p...,"[sebenernya, gak, kuat, allah, kalo, kuliah, p...",sebenernya gak kuat allah kalo kuliah pagi kua...
3,bungwol,2022-10-05 23:59:22+00:00,YaAllah w yg dulu kuliah PP tangsel rawamangun...,YaAllah w yg dulu kuliah PP tangsel rawamangun...,YaAllah w yg dulu kuliah PP tangsel rawamangun...,YaAllah w yg dulu kuliah PP tangsel rawamangun...,yaallah w yg dulu kuliah pp tangsel rawamangun...,"[yaallah, kuliah, pp, tangsel, rawamangun, aje...",yaallah kuliah pp tangsel rawamangun aje dah m...
4,newaccautoflbck,2022-10-05 23:59:18+00:00,ptn! kuliah di upnvj gapapa kan? https://t.co/...,ptn! kuliah di upnvj gapapa kan? https://t.co/...,ptn kuliah di upnvj gapapa kan,ptn kuliah di upnvj gapapa kan,ptn kuliah di upnvj gapapa kan,"[ptn, kuliah, upnvj, gapapa]",ptn kuliah upnvj gapapa
5,tapibohoonk,2022-10-05 23:59:15+00:00,[cm] kalian sering kehilangan ambis gak sih pa...,[cm] kalian sering kehilangan ambis gak sih pa...,cm kalian sering kehilangan ambis gak sih pas ...,kalian sering kehilangan ambis gak sih pas kul...,kalian sering kehilangan ambis gak sih pas kul...,"[hilang, ambis, gak, pas, kuliah, kayak, gak, ...",hilang ambis gak pas kuliah kayak gak semangat...
6,cutmybutter2fly,2022-10-05 23:59:14+00:00,TWITTER PLEASE DO YOUR MAGIC\n\nentah ini work...,TWITTEPLEASE DO YOUMAGIC\n\nentah ini work it ...,TWITTEPLEASE DO YOUMAGIC entah ini work it or ...,TWITTEPLEASE DO YOUMAGIC entah ini work it or ...,twitteplease do youmagic entah ini work it or ...,"[twitteplease, do, youmagic, work, it, or, not...",twitteplease do youmagic work it or not coba n...
7,bydamieee,2022-10-05 23:59:13+00:00,CLAP UDH ADA CHALLENGE NYA BLM SI??? PGN JOINN...,CLAP UDH ADA CHALLENGE NYA BLM SI??? PGN JOINN...,CLAP UDH ADA CHALLENGE NYA BLM SI PGN JOINNN S...,CLAP UDH ADA CHALLENGE NYA BLM SI PGN JOINNN S...,clap udh ada challenge nya blm si pgn joinnn s...,"[clap, challenge, blm, pgn, joinnn, sm, tmn, k...",clap challenge blm pgn joinnn sm tmn kuliah ke...
9,FukeinaKo,2022-10-05 23:58:53+00:00,Tidur jam 3. Bangun 6.20. Kuliah 6.30,Tidur jam 3. Bangun 6.20. Kuliah 6.30,Tidur jam 3 Bangun 6 20 Kuliah 6 30,Tidur jam Bangun Kuliah,tidur jam bangun kuliah,"[tidur, jam, bangun, kuliah]",tidur jam bangun kuliah
10,dibawahpressure,2022-10-05 23:58:44+00:00,Riiill. Dari aku SD sampe sekarang kuliah kebu...,Riiill. Dari aku SD sampe sekarang kuliah kebu...,Riiill Dari aku SD sampe sekarang kuliah kebun...,Riiill Dari aku SD sampe sekarang kuliah kebun...,riiill dari aku sd sampe sekarang kuliah kebun...,"[riiill, sd, sampe, kuliah, kebun, papa, modal]",riiill sd sampe kuliah kebun papa modal


In [14]:
ds = ds[ds['data'] != '']
ds.head()

,username,tweetcreatedat,text,clean_text,remove_symb,clean_text_two,case_fold,tokenize,data
0,PijaMoris,2022-10-05 23:59:59+00:00,@fedtriyahya Selawat yg ustaz wadi anuar selal...,Selawat yg ustaz wadi anuar selalu sebut dala...,Selawat yg ustaz wadi anuar selalu sebut dalam...,Selawat yg ustaz wadi anuar selalu sebut dalam...,selawat yg ustaz wadi anuar selalu sebut dalam...,"[selawat, ustaz, wadi, anuar, kuliah, hengat, ...",selawat ustaz wadi anuar kuliah hengat aih nam...
1,akaipinheel,2022-10-05 23:59:52+00:00,@collegemenfess siapa yang ngira dulu kalo mau...,siapa yang ngira dulu kalo mau jadi polisi ha...,siapa yang ngira dulu kalo mau jadi polisi har...,siapa yang ngira dulu kalo mau jadi polisi har...,siapa yang ngira dulu kalo mau jadi polisi har...,"[ngira, kalo, polisi, kuliah]",ngira kalo polisi kuliah
2,buyny,2022-10-05 23:59:52+00:00,sebenernya aku gak kuat ya Allah kalo kuliah p...,sebenernya aku gak kuat ya Allah kalo kuliah p...,sebenernya aku gak kuat ya Allah kalo kuliah p...,sebenernya aku gak kuat ya Allah kalo kuliah p...,sebenernya aku gak kuat ya allah kalo kuliah p...,"[sebenernya, gak, kuat, allah, kalo, kuliah, p...",sebenernya gak kuat allah kalo kuliah pagi kua...
3,bungwol,2022-10-05 23:59:22+00:00,YaAllah w yg dulu kuliah PP tangsel rawamangun...,YaAllah w yg dulu kuliah PP tangsel rawamangun...,YaAllah w yg dulu kuliah PP tangsel rawamangun...,YaAllah w yg dulu kuliah PP tangsel rawamangun...,yaallah w yg dulu kuliah pp tangsel rawamangun...,"[yaallah, kuliah, pp, tangsel, rawamangun, aje...",yaallah kuliah pp tangsel rawamangun aje dah m...
4,newaccautoflbck,2022-10-05 23:59:18+00:00,ptn! kuliah di upnvj gapapa kan? https://t.co/...,ptn! kuliah di upnvj gapapa kan? https://t.co/...,ptn kuliah di upnvj gapapa kan,ptn kuliah di upnvj gapapa kan,ptn kuliah di upnvj gapapa kan,"[ptn, kuliah, upnvj, gapapa]",ptn kuliah upnvj gapapa


In [15]:
ds.shape

(375, 9)

In [14]:
ds.to_csv('re-preprocessing.csv', encoding='utf8', index=False)

In [16]:
ds.drop(ds.columns[0:8], axis = 1, inplace = True)

In [17]:
ds.head()

,data
0,selawat ustaz wadi anuar kuliah hengat aih nam...
1,ngira kalo polisi kuliah
2,sebenernya gak kuat allah kalo kuliah pagi kua...
3,yaallah kuliah pp tangsel rawamangun aje dah m...
4,ptn kuliah upnvj gapapa


In [20]:
#penyimpanan ke csv
ds.to_csv('clean_data_perkuliahan.csv', encoding = 'utf8', index = False)

In [22]:
ds.shape

(375, 1)